In [1]:
#Preprocessing for neural network using GloVe embedding
#preprocessing packages
import pandas as pd
from nltk.stem import SnowballStemmer
import re
import glob
import spacy
import en_core_web_sm
import concurrent.futures
spacy.prefer_gpu()
nlp = en_core_web_sm.load()
stemmer=SnowballStemmer('english')

In [2]:
datalist=glob.glob('spacy_data/*.csv')

In [3]:
combined_datalist=[pd.read_csv(i) for i in datalist] 
combined_datalist=pd.concat([pd.DataFrame(combined_datalist[i]) for i in range(len(combined_datalist))], sort=True)

In [4]:
combined_datalist

,Unnamed: 0,index,text
0,24000,Management_of_Material_Resources.xls,"Description \n At Exelon, we've got a place fo..."
1,24001,Management_of_Material_Resources.xls,Responsibilities Execute internal and client f...
2,24002,Management_of_Material_Resources.xls,We are searching for a Senior IT Infrastructur...
3,24003,Management_of_Material_Resources.xls,Are you a seasoned Project Manager with experi...
4,24004,Management_of_Material_Resources.xls,Xylem |ˈzīləm| \n1) The tissue in plants that ...
...,...,...,...
5995,47995,Active_Learning.xls,Summary \n As a member of the ChapCare Medical...
5996,47996,Active_Learning.xls,POSITION OBJECTIVE/SUMMARY: Under the directi...
5997,47997,Active_Learning.xls,"Mobile Medical & Nursing, Inc was started by a..."
5998,47998,Active_Learning.xls,Perlman Clinic (www.perlmanclinic.com) is look...


In [4]:
rec_data=combined_datalist.fillna('None')

In [6]:
rec_data

,Unnamed: 0,index,text
0,24000,Management_of_Material_Resources.xls,"Description \n At Exelon, we've got a place fo..."
1,24001,Management_of_Material_Resources.xls,Responsibilities Execute internal and client f...
2,24002,Management_of_Material_Resources.xls,We are searching for a Senior IT Infrastructur...
3,24003,Management_of_Material_Resources.xls,Are you a seasoned Project Manager with experi...
4,24004,Management_of_Material_Resources.xls,Xylem |ˈzīləm| \n1) The tissue in plants that ...
...,...,...,...
5995,47995,Active_Learning.xls,Summary \n As a member of the ChapCare Medical...
5996,47996,Active_Learning.xls,POSITION OBJECTIVE/SUMMARY: Under the directi...
5997,47997,Active_Learning.xls,"Mobile Medical & Nursing, Inc was started by a..."
5998,47998,Active_Learning.xls,Perlman Clinic (www.perlmanclinic.com) is look...


In [5]:
combined_datalist.columns

Index(['Unnamed: 0', 'index', 'text'], dtype='object')

In [6]:
with open ('stopwords.txt', 'r') as f:
    word=f.read().split('\n')

word=word
f.close()

In [7]:
def clean_remove_emails_https(X_data):
    X_t=[]
    for i in X_data:
        X_t.append(' '.join([p.replace('\n',' ').replace('>','').replace('<','')\
         .replace('-',' ').lower()  for p in i.split()]))

    #remove emails and http(s) streams - can be removed
    collection=[]
    for i in X_t:
        collection.append(re.findall(r'[A-Za-z0-9_\.@+]{8,30}\.edu|[A-Za-z0-9\._@+]{8,30}\.com|\
                                     [A-Za-z0-9_\.@+]{8,30}\.gov|[A-Za-z0-9_\.@+]{8,30}\.net|\
                                     |http://[a-z0-9_\.@+/=?#]{1,150}|https://[a-z0-9_\.@+/=?#]{1,150}|\
                                     www[a-z0-9_\.@+]{8,150}|www[a-z0-9_\.@+]{8,150}', str(i)))
     
    X_emails__http_rm=[]
    for i,j in zip(X_t,collection):
        X_emails__http_rm.append(' '.join([p for p in i.split() if p not in j]))

    return X_emails__http_rm

In [8]:
%%time
X = rec_data.text
X_emails__http_rm_data=clean_remove_emails_https(X)

CPU times: user 32 s, sys: 328 ms, total: 32.3 s
Wall time: 32.3 s


In [25]:
def nouns_verbs_only(data):
    words_with_pos=[]
    for i in data:
        words_with_pos.append([[k.text, k.pos_] for k in nlp(i)])
    
    nouns_verbs_only=[]
    for i in words_with_pos:
        wrds=[k[0] for k in i]
        pos=[k[1] for k in i]
        df=pd.concat([pd.Series(wrds, name='words'),pd.Series(pos, name='pos')], axis=1)
        df=df[(df.pos=='NOUN')|(df.pos=='VERB')].reset_index().iloc[:,1:]
        nouns_verbs_only.append(df)
        
    nouns_verb_only_text=[list(i.words) for i in nouns_verbs_only]
    
       
    return nouns_verb_only_text,nouns_verbs_only

In [10]:
def nouns_only(data):
    words_with_pos=[]
    for i in data:
        words_with_pos.append([[k.text, k.pos_] for k in nlp(i)])
    
    nouns_only=[]
    for i in words_with_pos:
        wrds=[k[0] for k in i]
        pos=[k[1] for k in i]
        df=pd.concat([pd.Series(wrds, name='words'),pd.Series(pos, name='pos')], axis=1)
        df=df[(df.pos=='NOUN')].reset_index().iloc[:,1:]
        nouns_only.append(df)
        
    nouns_only_text=[list(i.words) for i in nouns_only]
    
       
    return nouns_only_text

In [11]:
def all_word_pos(data):
    words_with_pos=[]
    for i in data:
        words_with_pos.append([[k.text, k.pos_] for k in nlp(i)])
    
    all_pos=[]
    for i in words_with_pos:
        wrds=[k[0] for k in i]
        pos=[k[1] for k in i]
        df=pd.concat([pd.Series(wrds, name='words'),pd.Series(pos, name='pos')], axis=1)
        df=df.reset_index().iloc[:,1:]
        all_pos.append(df)
        
    #can subset df after run to focus desired positions
    return all_pos

In [12]:
'''
%%time
processed_text=[]
with concurrent.futures.ProcessPoolExecutor() as f:
    for i in f.map(all_word_pos, X_emails__http_rm_data):
        processed_text.append(i)
'''
pass

In [13]:
%%time 
#28 mins 56,003 rows data & Spacy GPU required (4 GPUs)
processed_text=nouns_verbs_only(X_emails__http_rm_data)

CPU times: user 28min 42s, sys: 3.18 s, total: 28min 45s
Wall time: 28min 45s


In [26]:
mini_processed_text=nouns_verbs_only(X_emails__http_rm_data[:5])

In [27]:
words=[]
pos=[]
for i in mini_processed_text[1]:
    words.append(i.words)
    pos.append(i.pos)

In [29]:
pos_dict=[]
for i,k in zip(pos,words):
    pos_dict.append(dict(zip(list(k),list(i))))


In [37]:
sample=pd.Series(pos_dict)
print(sample[0])

{'description': 'NOUN', 'got': 'VERB', 'place': 'NOUN', 'join': 'VERB', 'nation': 'NOUN', 'leading': 'VERB', 'energy': 'NOUN', 'provider': 'NOUN', 'electricity': 'NOUN', 'generation': 'NOUN', 'portfolios': 'NOUN', 'customer': 'NOUN', 'bases': 'NOUN', 'country': 'NOUN', 'will': 'VERB', 'part': 'NOUN', 'family': 'NOUN', 'companies': 'NOUN', 'strives': 'VERB', 'standards': 'NOUN', 'power': 'NOUN', 'sales': 'NOUN', 'delivery': 'NOUN', 'team': 'NOUN', 'professionals': 'NOUN', 'focused': 'VERB', 'performance': 'NOUN', 'thought': 'VERB', 'leadership': 'NOUN', 'innovation': 'NOUN', 'ideas': 'NOUN', 'come': 'VERB', 'workforce': 'NOUN', 'provide': 'VERB', 'tools': 'NOUN', 'resources': 'NOUN', 'need': 'VERB', 'design': 'NOUN', 'build': 'VERB', 'enhance': 'VERB', 'career': 'NOUN', 'motivating': 'VERB', 'success': 'NOUN', 'employees': 'NOUN', 'base': 'NOUN', 'salary': 'NOUN', 'incentives': 'NOUN', 'health': 'NOUN', 'retirement': 'NOUN', 'benefits': 'NOUN', 'share': 'VERB', 'passion': 'NOUN', 'think

In [19]:
from datetime import date

In [20]:
#rec_data.to_csv(f'processed_{date.today()}.csv')

In [31]:
def rm_stopword_stem_word_cnt(data):
       
    no_stopwords=[]
    for i in data:
        no_stopwords.append(' '.join([j for j in i if j not in word and len(j) >=3]))
        
    no_stop_stemmed_words=[]
    for p in no_stopwords:
        no_stop_stemmed_words.append(' '.join([stemmer.stem(k) for k in p.split()]))
    
    data_len=[]
    for i in no_stopwords:
        data_len.append(len(i.split()))
        
    return no_stop_stemmed_words,data_len

In [32]:
%%time
no_stop_stemmed=rm_stopword_stem_word_cnt(processed_text) 

CPU times: user 2min 59s, sys: 28 ms, total: 2min 59s
Wall time: 2min 59s


In [41]:
rec_data['no_stopwords_stemmed']=pd.Series(no_stop_stemmed[0])
rec_data['data_len']=pd.Series(no_stop_stemmed[1])
rec_data=rec_data.fillna('None')
data=rec_data
data2=data[(data.data_len>0)]
data3=data2[data2.data_len>data2.data_len.mean()].reset_index().iloc[:,1:]

In [42]:
data3.head()

,Unnamed: 0,index,text,no_stopwords_stemmed,data_len
0,24000,Management_of_Material_Resources.xls,"Description \n At Exelon, we've got a place fo...",descript place join nation lead energi provid ...,419
1,24004,Management_of_Material_Resources.xls,Xylem |ˈzīləm| \n1) The tissue in plants that ...,tissu plant bring water root lead water techno...,338
2,24006,Management_of_Material_Resources.xls,Requisition Number 1185 \n \n Classification...,requisit number classif titl grade job titl pr...,550
3,24008,Management_of_Material_Resources.xls,Project / Program Manager 3 \n Location: Carso...,project program manag locat durat month contra...,541
4,24009,Management_of_Material_Resources.xls,Our client is just starting their S4 Hana jour...,client start journey hire big4 consult firm sy...,273


In [ ]:
#data3.to_csv('occ_preprocessed_.csv')